# Overview

---

**BLAH BLAH BLAH**

---

# Libraries and Functions

In [1]:
# Import Libraries needed by the Lambda Function
import numpy as np
import h5py
import scipy
import os
from os import environ
import json
from json import dumps, loads
from boto3 import client, resource, Session
import botocore
import uuid
import io
from redis import StrictRedis as redis

# Import libraries needed for the Codebook
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Global Variables
s3_client = client('s3', region_name='us-west-2') # S3 access
s3_resource = resource('s3')
redis_client = client('elasticache', region_name='us-west-2')
lambda_client = client('lambda', region_name='us-west-2') # Lambda invocations
# Retrieve the Elasticache Cluster endpoint
cc = redis_client.describe_cache_clusters(ShowCacheNodeInfo=True)
endpoint = cc['CacheClusters'][0]['CacheNodes'][0]['Endpoint']['Address']
cache = redis(host=endpoint, port=6379, db=0)

In [3]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))

    return s

In [4]:
def vectorize(x_orig):
    """
    Vectorize the image data into a matrix of column vectors
    
    Argument:
    x_orig -- Numpy array of image data
    
    Return:
    Reshaped/Transposed Numpy array
    """
    return x_orig.reshape(x_orig.shape[0], -1).T

In [5]:

def standardize(x_orig):
    """
    Standardize the input data
    
    Argument:
    x_orig -- Numpy array of image data
    
    Return:
    Call to `vectorize()`, stndrdized Numpy array of image data
    """
    return vectorize(x_orig) / 255

In [6]:
def initialize_data(endpoint, w, b):
    """
    Extracts the training and testing data from S3, flattens, 
    standardizes and then dumps the data to ElastiCache 
    for neurons to process as layer a^0
    """
    
    # Load main dataset
    dataset = h5py.File('/tmp/datasets.h5', "r")
    
    # Create numpy arrays from the various h5 datasets
    train_set_x_orig = np.array(dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(dataset["train_set_y"][:]) # train set labels
    test_set_x_orig = np.array(dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(dataset["test_set_y"][:]) # test set labels
    
    # Reshape labels
    train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    # Preprocess inputs
    train_set_x = standardize(train_set_x_orig)
    test_set_x = standardize(test_set_x_orig)

    # Dump the inputs to the temporary s3 bucket for TrainerLambda
    #bucket = storage_init() # Creates a temporary bucket for the propogation steps
    data_keys = {} # Dictionary for the hask keys of the data set
    dims = {} # Dictionary of data set dimensions
    a_list = [train_set_x, train_set_y, test_set_x, test_set_y]
    a_names = [] # Placeholder for array names
    for i in range(len(a_list)):
        # Create a lis of the names of the numpy arrays
        a_names.append(name2str(a_list[i], locals()))
    for j in range(len(a_list)):
        # Dump the numpy arrays to ElastiCache
        data_keys[str(a_names[j][0])] = to_cache(endpoint, obj=a_list[j], name=a_names[j][0])
        # Append the array dimensions to the list
        dims[str(a_names[j][0])] = a_list[j].shape
    
    # Initialize weights
    if w == 0: # Initialize weights to dimensions of the input data
        dim = dims.get('train_set_x')[0]
        weights = np.zeros((dim, 1))
        # Store the initial weights as a column vector on S3
        data_keys['weights'] = to_cache(endpoint, obj=weights, name='weights')
    else:
        #placeholder for random weight initialization
        pass
        
    # Initialize Bias
    if b != 0:
        #placeholder for random bias initialization
        #data_keys['bias'] = to_cache(endpoint, obj=bias, name='bias')
        pass
    else:
        data_keys['bias'] = to_cache(endpoint, obj=b, name='bias')
    
    # Initialize training example size
    m = train_set_x.shape[1]
    data_keys['m'] = to_cache(endpoint, obj=m, name='m')
    
#    # Initialize the results tracking object
#    to_cache(endpoint, dump='', name='results')
        
    return data_keys, [j for i in a_names for j in i], dims

In [7]:
def to_cache(endpoint, obj, name):
    """
    Serializes multiple data type to ElastiCache and returns
    the Key.
    
    Arguments:
    endpoint -- The ElastiCache endpoint
    obj -- the object to srialize. Can be of type:
            - Numpy Array
            - Python Dictionary
            - String
            - Integer
    name -- Name of the Key
    
    Returns:
    key -- For each type the key is made up of {name}|{type} and for
           the case of Numpy Arrays, the Length and Widtch of the 
           array are added to the Key.
    """
    if 'numpy' in str(type(obj)):
        array_dtype = str(obj.dtype)
        length, width = obj.shape
        # Convert the array to string
        val = obj.ravel().tostring()
        # Create a key from the name and necessary parameters from the array
        # i.e. {name}|{type}#{length}#{width}
        key = '{0}|{1}#{2}#{3}'.format(name, array_dtype, length, width)
        # Store the binary string to Redis
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    elif type(obj) is str:
        key = '{0}|{1}'.format(name, 'string')
        val = obj
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    elif type(obj) is int:
        key = '{0}|{1}'.format(name, 'int')
        val = str(obj)
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    elif type(obj) is dict:
        #x = json.dumps(obj)
        #val = json.loads(x)
        val = json.dumps(obj)
        key = '{0}|{1}'.format(name, 'json')
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key

In [8]:
def from_cache(endpoint, key):
    """
    De-serializes binary object from ElastiCache by reading
    the type of object from the name and converting it to
    the appropriate data type.
    
    Arguments:
    endpoint -- ElastiCache endpoint.
    key -- Name of the Key to retrieve the object.
    
    Returns:
    obj -- The object converted to specifed data type.
    """
    
    # Check if the Key is for a Numpy array containing
    # `float64` data types
    if 'float64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a Numpy array containing
    # `int64` data types
    elif 'int64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        data = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        obj = np.fromstring(data, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a json type
    elif 'json' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return json.loads(obj)
    # Chec if the Key is an integer
    elif 'int' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return int(obj)
    # Check if the Key is a string
    elif 'string' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return obj


In [9]:
def name2str(obj, namespace):
    """
    Converts the name of the numpy array to string
    
    Arguments:
    obj -- Numpy array object
    namespace -- dictionary of the current global symbol table
    
    Return:
    List of the names of the Numpy arrays
    """
    return [name for name in namespace if namespace[name] is obj]

---
# Launch Event

In [10]:
w = 0
b = 0
# Simulate S3 event trigger data
event = {
    "Records": [
        {
            "eventVersion": "2.0",
            "eventTime": "1970-01-01T00:00:00.000Z",
            "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
             },
            "s3": {
                "configurationId": "testConfigRule",
                "object": {
                    "eTag": "0123456789abcdef0123456789abcdef",
                    "sequencer": "0A1B2C3D4E5F678901",
                    "key": "training_input/datasets.h5",
                    "size": 1024
                },
                "bucket": {
                    "arn": "arn:aws:s3:::lnn",
                    "name": "lnn",
                    "ownerIdentity": {
                        "principalId": "EXAMPLE"
                    }
                },
                "s3SchemaVersion": "1.0"
            },
            "responseElements": {
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH",
                "x-amz-request-id": "EXAMPLE123456789"
            },
            "awsRegion": "us-west-2",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "EXAMPLE"
            },
            "eventSource": "aws:s3"
        }
    ]
}

# Simulate TrainerLambda ARN
#environ[str('TrainerLambda')] = str(None)

In [11]:
# Retrieve datasets and setting from S3
input_bucket = s3_resource.Bucket(str(event['Records'][0]['s3']['bucket']['name']))
dataset_key = str(event['Records'][0]['s3']['object']['key'])
settings_key = dataset_key.split('/')[-2] + '/parameters.json'
try:
    input_bucket.download_file(dataset_key, '/tmp/datasets.h5')
    input_bucket.download_file(settings_key, '/tmp/parameters.json')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == '404':
        print("Error downloading input data from S3, S3 object does not exist")
    else:
        raise
    
# Extract the neural network parameters
with open('/tmp/parameters.json') as parameters_file:
    parameters = json.load(parameters_file)
    
# Build in additional parameters from neural network parameters
parameters['epoch'] = 1
# Next Layer to process
parameters['layer'] = 1
# Input data sets and data set parameters
parameters['data_keys'], parameters['input_data'], parameters['data_dimensions'] = initialize_data(endpoint=endpoint, w=parameters.get('weight'), b=parameters.get('bias'))
    
# Initialize payload to `TrainerLambda`
payload = {}
# Initialize the overall state
payload['state'] = 'start'
# Dump the parameters to ElastiCache
payload['parameter_key'] = to_cache(endpoint, obj=parameters, name='parameters')
#payload['endpoint'] = endpoint
# Prepare the payload for `TrainerLambda`
payloadbytes = dumps(payload)
    
print("Complete Neural Network Settings: \n")
print(dumps(parameters, indent=4, sort_keys=True))
print("Payload to be sent to TrainerLambda: \n" + dumps(payload, indent=4, sort_keys=True))

Complete Neural Network Settings: 

{
    "activations": {
        "layer1": "sigmoid"
    },
    "bias": 0,
    "data_dimensions": {
        "test_set_x": [
            12288,
            50
        ],
        "test_set_y": [
            1,
            50
        ],
        "train_set_x": [
            12288,
            209
        ],
        "train_set_y": [
            1,
            209
        ]
    },
    "data_keys": {
        "bias": "bias|int",
        "m": "m|int",
        "test_set_x": "test_set_x|float64#12288#50",
        "test_set_y": "test_set_y|int64#1#50",
        "train_set_x": "train_set_x|float64#12288#209",
        "train_set_y": "train_set_y|int64#1#209",
        "weights": "weights|float64#12288#1"
    },
    "epoch": 1,
    "epochs": 1,
    "input_data": [
        "train_set_x",
        "train_set_y",
        "test_set_x",
        "test_set_y"
    ],
    "layer": 1,
    "layers": 1,
    "learning_rate": 0.005,
    "neurons": {
        "layer1": 1
    },
    "we

---
# Trainer -> Neuron Event
**Simulating Forward Propogation of the Neuron**

In [12]:
event = payload

In [13]:
global parameter_key
parameter_key = event.get('parameter_key')
global parameters 
parameters = from_cache(endpoint, parameter_key)

In [14]:
w = from_cache(endpoint=endpoint, key=parameters['data_keys']['weights'])
b = from_cache(endpoint=endpoint, key=parameters['data_keys']['bias'])
X = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_x'])
Y = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_y'])
m = from_cache(endpoint=endpoint, key=parameters['data_keys']['m'])

In [15]:
a = sigmoid(np.dot(w.T, X) + b)

In [16]:
a.shape

(1, 209)

In [17]:
a

array([[ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5, 

In [18]:
ID = 1
to_cache(endpoint=endpoint, obj=a, name='a_'+str(ID))

'a_1|float64#1#209'

---
# Neuron -> Trainer Event
**Simulating processing the Cost from the Activation output/s**

In [88]:
for key in cache.scan_iter(match='a_*'):
    print(key)

b'a_1|float64#1#209'
b'a_2|float64#1#209'


In [89]:
r = redis(host=endpoint, port=6379, db=0, charset="utf-8", decode_responses=True)
for key in r.scan_iter(match='a_*'):
    print(key)

a_1|float64#1#209
a_2|float64#1#209


In [86]:
A = np.array([])
r = redis(host=endpoint, port=6379, db=0, charset="utf-8", decode_responses=True)
for key in r.scan_iter(match='a_*'):
    B = from_cache(endpoint, key)
    C = np.append(A, B)

In [87]:
print(C.shape)

(209,)


In [23]:
B = from_cache(endpoint, key='a_1|float64#1#209')

In [24]:
C = np.append(A, B)

In [25]:
C

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0

In [26]:
B.shape

(1, 209)

In [27]:
C.shape

(209,)

In [28]:
C = C.reshape(1, B.shape[1])

In [29]:
C.shape

(1, 209)

In [41]:
C

array([[ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5, 

In [42]:
assert(C.shape == B.shape)

---

---

---

---
**Merging two activation arrays**

In [43]:
key = 'a_1|float64#1#209'
A = from_cache(endpoint, key=key)
B = from_cache(endpoint, key=key)
assert(A.shape == B.shape)

In [44]:
C = np.vstack((A, B))
C.shape

(2, 209)

In [45]:
C

array([[ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
         0.5,  0.5,  0.5,  0.5,  0.5, 

---

---

**Merging three activation arrays**

In [52]:
A = np.array([]).reshape(0, count)
A.shape

(0, 2)

In [53]:
key1 = 'a_1|float64#1#209'
key2 = 'a_2|float64#1#209'
a = from_cache(endpoint, key=key1)
b = from_cache(endpoint, key=key2)
c = from_cache(endpoint, key=key1)
assert(a.shape == b.shape)

In [54]:
A = np.concatenate((a, b))

In [55]:
A.shape

(2, 209)

In [56]:
A = np.concatenate((A, c))

In [58]:
A.shape

(3, 209)

**Thought Process on how to the above programatically** 

In [79]:
to_concat = []
r = redis(host=endpoint, port=6379, db=0, charset="utf-8", decode_responses=True)
count = 0
index = []
for key in r.scan_iter(match='a_*'):
    to_concat.append(key)
    index.append(count)
    count = count + 1
print(count)
print(to_concat)
print(index)

2
['a_1|float64#1#209', 'a_2|float64#1#209']
[0, 1]


In [80]:
for i in range(count):
    index[i] = from_cache(endpoint=endpoint, key=to_concat[i])

In [81]:
index

[array([[ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
          0.5,  0.5,  0.5